In [1]:
import sys, os
import pandas as pd
import numpy as np


def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from keras.models import Model
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import roc_auc_score

Using TensorFlow backend.


In [2]:
import gc
gc.enable()
gc.collect()

0

In [34]:
gc.collect()

0

In [19]:
random_seed = 213
np.random.seed(random_seed)
noise_std = 0.01

# import data
train = pd.read_csv("/Users/JoonH/dont-overfit-ii/train.csv")
train_y = train['target']
train_X = train.drop(['id','target'], axis=1).values

test = pd.read_csv("/Users/JoonH/dont-overfit-ii/test.csv")
test = test.drop(['id'], axis=1).values

# scale using RobustScaler
# fitting scaler on full data outperforms fitting on test_X only (+0.006 kaggle score)
data = RobustScaler().fit_transform(np.concatenate((train_X, test), axis=0))
train_X = data[:250]
test = data[250:]

#scale only with respect to training data to prevent leakage
#scaler = RobustScaler().fit(train_X)
#train_X = scaler.transform(train_X)
#test = scaler.transform(test)

# add a bit of noise to train_X to reduce overfitting
train_X += np.random.normal(0, noise_std, train_X.shape)

In [32]:
def get_model():
    K.clear_session()
    inputs = Input(shape=(300,))
    x = Dense(64, activation=LeakyReLU(alpha = 0.1), kernel_regularizer = 'l1')(inputs)
    x = Dropout(rate = 0.5) (x)
    x = Dense(64, activation=LeakyReLU(alpha = 0.1), kernel_regularizer = 'l1')(inputs)
    x = Dropout(rate = 0.5) (x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [25]:
#https://www.kaggle.com/c/mercedes-benz-greener-manufacturing/discussion/34019
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [33]:
counter = 0
sss_n_splits = 20
sss_test_size = 0.35
predictions = pd.DataFrame()
r2_threshold = 0.185
for train_index, val_index in StratifiedShuffleSplit(n_splits=sss_n_splits, test_size=sss_test_size, random_state=random_seed).split(train_X, train_y):
    X, val_X = train_X[train_index], train_X[val_index]
    y, val_y = train_y[train_index], train_y[val_index]
    K.clear_session()
    model = get_model()
    early = EarlyStopping(patience = 3, monitor = 'val_loss', verbose = 0)
    
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[r2_keras])
    model.fit(X, y, epochs = 200, batch_size = 64, validation_data=(val_X, val_y), shuffle = True, verbose = 0, callbacks = [early])
    
    val_loss, val_r2 = model.evaluate(val_X, val_y, verbose=0)
    val_y_pred = model.predict(val_X)
    val_roc = roc_auc_score(val_y, val_y_pred)
    
    
    if (val_r2 > r2_threshold):
        counter+=1
        print( str(counter) + "  validation loss: " + str(val_loss) + ', ROC_AUC: ' + str(val_roc) + ', R2: ' + str(val_r2) + ' <-- OK')
        prediction = model.predict(test)
        predictions = pd.concat([predictions, pd.DataFrame(prediction)], axis=1)
        
    #else:
        #print("validation loss: " + str(val_loss) + ' ROC_AUC: ' + str(val_roc) + 'R2: ' + str(val_r2) + ' <-- Skipping')
        
print(str(counter) + ' out of ' + str(sss_n_splits) + ' chosen for ensemble')
mean_pred = pd.DataFrame(predictions.mean(axis=1))
mean_pred.index += 250
mean_pred.columns = ['target']
mean_pred.to_csv('/Users/JoonH/dont_overfit2_mlp.csv', index_label='id', index=True)    

1  validation loss: 0.7851362553509799, ROC_AUC: 0.8214285714285715, R2: 0.2162791598926891 <-- OK
2  validation loss: 0.7862817482514814, ROC_AUC: 0.7762276785714286, R2: 0.17229208621111783 <-- OK
3  validation loss: 0.799250451001254, ROC_AUC: 0.7829241071428571, R2: 0.16437530517578125 <-- OK
4  validation loss: 0.7427422404289246, ROC_AUC: 0.8683035714285715, R2: 0.25172948295419867 <-- OK
5  validation loss: 0.7652633569457314, ROC_AUC: 0.80859375, R2: 0.2085599357431585 <-- OK
6  validation loss: 0.7609402699904009, ROC_AUC: 0.8169642857142857, R2: 0.1726505160331726 <-- OK
7  validation loss: 0.7242300943894819, ROC_AUC: 0.8805803571428571, R2: 0.27306615764444525 <-- OK
8  validation loss: 0.7711965224959634, ROC_AUC: 0.8046875, R2: 0.21149024096402255 <-- OK
9  validation loss: 0.755522608757019, ROC_AUC: 0.8727678571428571, R2: 0.19667383215644144 <-- OK
10  validation loss: 0.7937669537284158, ROC_AUC: 0.7700892857142858, R2: 0.1643953648480502 <-- OK
11  validation loss: 0